In [ ]:
## file loading

from google.colab import drive
import shutil
import zipfile

drive.mount('/content/drive')

shutil.copy('/content/drive/My Drive/data/leafdry_with_csv.zip', '/content')

zip_file_train_img='/content/leafdry_with_csv.zip'

with zipfile.ZipFile(zip_file_train_img, 'r') as zip_ref:
  zip_ref.extractall('/content/leafdry')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Annotation 생성작업

In [ ]:
import pandas as pd
import numpy as np
import os
import re

In [ ]:
#한글이 포함된 라벨링 데이터 및 파일명을 영문으로 변환

In [ ]:
df_train = pd.read_csv('/content/leafdry/csv_files/leafdry_train_label_preprocessed.csv', encoding='utf-8')
df_test = pd.read_csv('/content/leafdry/csv_files/leafdry_test_label_preprocessed.csv', encoding='utf-8')
df_validation = pd.read_csv('/content/leafdry/csv_files/leafdry_validation_label_preprocessed.csv', encoding='utf-8')
df_model_validation = pd.read_csv('/content/leafdry/csv_files/leafdry_model_validation_label_preprocessed.csv', encoding='utf-8')

df_train['img_kor'] = df_train['img']
df_test['img_kor'] = df_test['img']
df_validation['img_kor'] = df_validation['img']
df_model_validation['img_kor'] = df_model_validation['img']

In [ ]:
##train

## 라벨링 데이터에서 한글이 포함된 문자열을 모두 영어로 바꾸기

df_train['img']=df_train['img'].apply(lambda x: x.replace('딸기','berry'))
df_train['img']=df_train['img'].apply(lambda x: x.replace('설향','snow'))
df_train['img']=df_train['img'].apply(lambda x: x.replace('잎끝마름','dry'))
df_train['class']=df_train['class'].apply(lambda x: x.replace('잎끝마름','dry'))
df_train['method']=df_train['method'].apply(lambda x: x.replace('칼슘부족','low_Ca'))

def replace_text(cell_value):
    if isinstance(cell_value, str):
      cell_value=cell_value.replace('줄기','stem')
      cell_value=cell_value.replace('잎','leaf')
      cell_value=cell_value.replace('과실','fruit')
      cell_value=cell_value.replace('화방','flower')
    return cell_value

df_train[df_train.filter(like='name_').columns]=df_train.filter(like='name_').applymap(replace_text)

In [ ]:
df_train.head()

,img,class,method,bbox_0,status_0,name_0,bbox_1,status_1,name_1,bbox_2,...,bbox_12,status_12,name_12,bbox_13,status_13,name_13,bbox_14,status_14,name_14,img_kor
0,berry_snow_dry_88_011_221128101701.jpg,dry,low_Ca,"[981.56, 759.33, 195.39, 162.18]",N,fruit,"[1060.33, 729.79, 152.72, 72.5]",Y,fruit,"[1088.79, 577.77, 105.67, 92.18]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,딸기_설향_잎끝마름_88_011_221128101701.jpg
1,berry_snow_dry_88_011_221129131701.jpg,dry,low_Ca,"[219.96, 513.36, 352.22, 111.62]",N,stem,"[121.22, 750.02, 369.81, 258.93]",N,leaf,"[994.01, 770.45, 196.73, 144.29]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,딸기_설향_잎끝마름_88_011_221129131701.jpg
2,berry_snow_dry_88_011_221129121701.jpg,dry,low_Ca,"[217.89, 512.58, 352.11, 108.32]",N,stem,"[121.39, 746.72, 370.14, 261.13]",N,leaf,"[991.01, 766.71, 199.08, 144.3]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,딸기_설향_잎끝마름_88_011_221129121701.jpg
3,berry_snow_dry_88_011_221126121701.jpg,dry,low_Ca,"[178.52, 747.01, 363.07, 268.2]",N,leaf,"[262.87, 499.04, 324.27, 125.38]",N,stem,"[978.56, 741.95, 218.56, 178.8]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,딸기_설향_잎끝마름_88_011_221126121701.jpg
4,berry_snow_dry_88_011_221130121701.jpg,dry,low_Ca,"[198.99, 522.69, 308.72, 125.89]",N,stem,"[96.17, 762.27, 366.97, 268.26]",N,leaf,"[1006.82, 791.7, 167.85, 145.24]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,딸기_설향_잎끝마름_88_011_221130121701.jpg


In [ ]:
df_train.to_csv('leafdry_train_en.csv',index=False)

In [ ]:
df_train_en=pd.read_csv('leafdry_train_en.csv',encoding='utf-8')

In [ ]:
## 실제 이미지 데이터 파일 이름도 라벨링 데이터 정보에 따라 변경하기
  ## 코랩에 파일 업로드시 한글파일명은 깨지므로, 이미지 파일명 변경은 로컬 jupyter notebook에서 작업함(아래 코드는 참고용)
# folder_path='/content/leafdry/train/train_img'

# for index, row in df_train_en.iterrows():
#     old_file_path=os.path.join(folder_path, row['img_kor'])
#     new_file_path=os.path.join(folder_path, row['img'])
#     try:
#         os.rename(old_file_path, new_file_path)
#         print(f'Renamed: {old_file_path} to {new_file_path}')
#     except FileNotFoundError:
#         print(f'File not found: {old_file_path}')
#     except FileExistsError:
#         print(f'File already exists: {new_file_path}')

In [ ]:
##test

## 라벨링 데이터에서 한글이 포함된 문자열을 모두 영어로 바꾸기

df_test['img']=df_test['img'].apply(lambda x: x.replace('딸기','berry'))
df_test['img']=df_test['img'].apply(lambda x: x.replace('설향','snow'))
df_test['img']=df_test['img'].apply(lambda x: x.replace('잎끝마름','dry'))
df_test['class']=df_test['class'].apply(lambda x: x.replace('잎끝마름','dry'))
df_test['method']=df_test['method'].apply(lambda x: x.replace('칼슘부족','low_Ca'))

def replace_text(cell_value):
    if isinstance(cell_value, str):
      cell_value=cell_value.replace('줄기','stem')
      cell_value=cell_value.replace('잎','leaf')
      cell_value=cell_value.replace('과실','fruit')
      cell_value=cell_value.replace('화방','flower')
    return cell_value

df_test[df_test.filter(like='name_').columns]=df_test.filter(like='name_').applymap(replace_text)

In [ ]:
##validation

## 라벨링 데이터에서 한글이 포함된 문자열을 모두 영어로 바꾸기

df_validation['img']=df_validation['img'].apply(lambda x: x.replace('딸기','berry'))
df_validation['img']=df_validation['img'].apply(lambda x: x.replace('설향','snow'))
df_validation['img']=df_validation['img'].apply(lambda x: x.replace('잎끝마름','dry'))
df_validation['class']=df_validation['class'].apply(lambda x: x.replace('잎끝마름','dry'))
df_validation['method']=df_validation['method'].apply(lambda x: x.replace('칼슘부족','low_Ca'))

def replace_text(cell_value):
    if isinstance(cell_value, str):
      cell_value=cell_value.replace('줄기','stem')
      cell_value=cell_value.replace('잎','leaf')
      cell_value=cell_value.replace('과실','fruit')
      cell_value=cell_value.replace('화방','flower')
    return cell_value

df_validation[df_validation.filter(like='name_').columns]=df_validation.filter(like='name_').applymap(replace_text)

In [ ]:
##model_validation

## 라벨링 데이터에서 한글이 포함된 문자열을 모두 영어로 바꾸기

df_model_validation['img']=df_model_validation['img'].apply(lambda x: x.replace('딸기','berry'))
df_model_validation['img']=df_model_validation['img'].apply(lambda x: x.replace('설향','snow'))
df_model_validation['img']=df_model_validation['img'].apply(lambda x: x.replace('잎끝마름','dry'))
df_model_validation['class']=df_model_validation['class'].apply(lambda x: x.replace('잎끝마름','dry'))
df_model_validation['method']=df_model_validation['method'].apply(lambda x: x.replace('칼슘부족','low_Ca'))

def replace_text(cell_value):
    if isinstance(cell_value, str):
      cell_value=cell_value.replace('줄기','stem')
      cell_value=cell_value.replace('잎','leaf')
      cell_value=cell_value.replace('과실','fruit')
      cell_value=cell_value.replace('화방','flower')
    return cell_value

df_model_validation[df_model_validation.filter(like='name_').columns]=df_model_validation.filter(like='name_').applymap(replace_text)

In [ ]:
df_test.to_csv('leafdry_test_en.csv',index=False)
df_validation.to_csv('leafdry_validation_en.csv',index=False)
df_model_validation.to_csv('leafdry_model_validation_en.csv',index=False)